# Suggestions Quick Start

In this section, we'll quickly set up suggestions for use in Pre Annotation in LightTag. By the end of this process you should have something that looks like this :  


![pre-annotations](./img/pre-annotations2.gif)

As with all of the other examples, we'll be interacting with LightTag's API using the  LTSession convenience class, which you can [download here](https://gist.github.com/talolard/793563397c48dca32f75c9d4b6f8f560)






In [4]:
from ltsession import LTSession 
from pprint import pprint # To print things pretty
session = LTSession(workspace="demo",user="lighttag",pwd="Shiva666")




In this quick start we'll assume you already have a Dataset and Schema defined and want to pre annotate the Dataset with Tags from the Schema. 
To do so, we'll 

1. Retreive our Dataset and Schema from LightTag 
2. Register a new  model 
3. Create the suggestions
4. Upload the suggestions
5. Attach the Model to a Task so that it's suggestions are shown to your annotators
6. Review your models output in LightTag's review mode



## 1. Fetching the dataset and the schema


In [5]:
dataset = session.get('v1/projects/default/datasets/bible/').json() #Use the slug of the dataset to fetch it from the datasets endpoint
examples = session.get('v1/projects/default/datasets/bible/examples/').json() #Use the slug of the dataset to fetch it from the datasets endpoint
schema = session.get('v1/projects/default/schemas/ner/').json() #Use the slug of the schema to fetch it from the schemas endpoint
pprint(examples[0])

{'content': ' And when thou hast stayed three days, then thou shalt go down '
            'quickly, and come to the place where thou didst hide thyself when '
            'the business was in hand, and shalt remain by the stone Ezel.  ',
 'dataset': '2a6e0c06-017d-473c-b3d5-bbb24c28b71c',
 'id': '259b4134-9d37-4b0e-86f7-37ec1f00a435',
 'metadata': {'book': 'The First Book of the Kings',
              'chapter': 20.179,
              'verse': 19}}


## 2. Registering a SuggestionModel
A SuggestionModel (or model for short) in LightTag is a container that contains all of the suggestions that came from a single source. Typically, a model corresponds to some ML model, dictionary or regular expressions that you have. 

* Models belong to a particular Schema
* Models can provide suggestions with Tags from that Schema **only**.  
* A single Model may not have overlapping suggestions. 
* You can have multiple Models in the same Schema and they can conflict

We'll use the new LightTag Model endpoint to register our model

In [7]:
model_definition = {
    'schema':schema['id'],
    'name':'My New New LightTag Suggestions Model',
    'metadata':{
        'anything':'here' #You can add an arbitrary JSON of metadata to your model 
    }
    
}
response = session.post('unstable/models/',json=model_definition)



LightTag will return the model object, including the schema id and the list of tags it is allowed to use


In [8]:

model = response.json()
pprint(model)

{'metadata': {'anything': 'here'},
 'name': 'My New New LightTag Suggestions Model',
 'schema': 'e7e7de79-1623-4803-9aac-7e664f12117a',
 'tags': [{'description': '',
           'id': '43ce3951-7ea1-423a-869f-ab07848d7f55',
           'name': 'PERSON'},
          {'description': '',
           'id': 'e332528b-dcb6-4414-9d14-fa61b12ab127',
           'name': 'LOCATION'},
          {'description': '',
           'id': '6e323f1d-1aa0-4e09-b64f-1ec0cd6dbed8',
           'name': 'ORGANIZATION'},
          {'description': '',
           'id': '978dfe2d-fdc3-4ca7-80d9-2baa0472f818',
           'name': 'TITLE'}],
 'url': 'https://demo.lighttag.io/api/unstable/models/72499576-01dc-4c1d-a6e0-7068ba11bb34/'}


## 3. Create your suggestions 
This step is mostly independent of LightTag. You need to create a list of objects that represent your suggestions. Each object should have 

1. **example_id** The LightTag provided id of the example you are suggesting on
2. **tag** OR **tag_id** The name of the tag (PERSON) or the LightTag id of the Tag
3. **start** The start offset of the text you are suggesting on 
4. **end** The end offset of the text you are suggesting on

#### Example
Below, we'll create a few suggestions using [Spacy](https://spacy.io/) a popular NLP library with built in NER


In [9]:
import spacy 
nlp = spacy.load("en_core_web_sm")

def process_example_with_spacy_and_return_lighttag_suggestions(example:dict):
    '''
        Example function that shows how to create suggestions for LightTag using Spacy. 
        The way you get suggestions will vary, but the output format is the same, a list of dicts
        with the properties start,end,tag and example_id

        This function takes an Example (as returned from LightTag's API) as input. 
        The example contains the id and content fields as well as metadata you may have uploaded with it
    '''
    content = example['content']
    doc = nlp(content) # Run spacy on the content of the example
    suggestions = [] # Empty list to store results
    for entity in doc.ents: # Spacy exposes the ents property which has named entities it found
        start = entity.start_char
        end = entity.end_char
        text = content[start:end] # Not required, but useful to see what your model is outputting
        suggestion = {
            'example_id':example['id'], # Then LightTag example_id
            'start':start, # The start offset of the span
            'end':end, # The end offset of the span 
            'tag':entity.label_, # The name of the tag being applied to the span
            'text':text # Not required, but useful to see what your model is outputting

        }

        suggestions.append(suggestion)
    return suggestions


In [10]:
suggestions = process_example_with_spacy_and_return_lighttag_suggestions(examples[2])
pprint(suggestions)

[{'end': 13,
  'example_id': '040c613d-df7f-4022-ad03-efeb42b3bf11',
  'start': 5,
  'tag': 'PERSON',
  'text': 'Jonathan'},
 {'end': 26,
  'example_id': '040c613d-df7f-4022-ad03-efeb42b3bf11',
  'start': 21,
  'tag': 'PERSON',
  'text': 'David'}]


## 4. Upload Your Suggestions
Now that we have suggestions, we upload them to the model. 
When you registered a model, LightTag responded with the Model object which includes it's URL. We'll post our suggestions there.

In [9]:
session.post(model['url']+'suggestions/',json=suggestions)

<Response [201]>

If all went well, you'll get back a 201 response. Let's **get** those suggestions 


In [10]:
resp = session.get(model['url']+'suggestions/',)
pprint(resp.json())

[{'end': 26,
  'example_id': '040c613d-df7f-4022-ad03-efeb42b3bf11',
  'id': '6b663fbd-1e7e-4ed4-92cc-d2ef78fd9c00',
  'start': 21,
  'tag': 'Tag PERSON from Schema: NER',
  'tag_id': '43ce3951-7ea1-423a-869f-ab07848d7f55',
  'value': 'David'},
 {'end': 13,
  'example_id': '040c613d-df7f-4022-ad03-efeb42b3bf11',
  'id': '4ef3f105-af4f-4ea5-8333-926700591f85',
  'start': 5,
  'tag': 'Tag PERSON from Schema: NER',
  'tag_id': '43ce3951-7ea1-423a-869f-ab07848d7f55',
  'value': ' Jonathan'}]


## 5. Attach Your Suggestions to a Task
In order to display your suggestions to your annotators, you need to explictly tell LightTag to do so. 
This is done by attaching the model to your task, which you can do either through the API or the UI. We'll show both ways here


#### 5.1 Attaching a model to an existing task in the UI

1. Go to the tasks section in the LightTag UI
  ![taskSection](./img/tasksSection.png)

2. Find the task, make sure it's on the same schema and dataset that your model is operating on 
  ![taskNoModel](./img/taskNoModel.png)
  
3. Open the Models section and select the model you just created
  ![taskWithModel](./img/taskWithModel.png)

#### 5.2 Attaching a model to an existing task with the API


In [11]:
# Retreive the tasks from LightTag API 
allTasks = session.get('v1/projects/default/task_definitions').json()
pprint(allTasks[0])



{'active': True,
 'allow_suggestions': True,
 'annotators_per_example': 3,
 'archived': False,
 'async_status': 'done',
 'complete_tasks': 0,
 'complete_tasksets': 0,
 'created_at': '2019-10-17T09:07:19.794526Z',
 'dataset_id': '2a6e0c06-017d-473c-b3d5-bbb24c28b71c',
 'guidelines': None,
 'id': '71b9280d-c132-4cda-a847-97878e1a4b46',
 'name': 'My Suggestions Task',
 'priority': 1,
 'progress': 0.0,
 'project_id': '2753ca38-69d9-4c96-9d31-df6d4069b027',
 'relationSchema_id': None,
 'remaining_tasks': 885,
 'schema_id': 'e7e7de79-1623-4803-9aac-7e664f12117a',
 'slug': 'my-suggestions-task',
 'status': 'active',
 'suggestion_models': ['9c26b461-c2ab-40ab-a2dc-cc454abe7d39'],
 'teams': ['c0d457f3-2609-4d1c-946b-7ab1dc72796a'],
 'total_tasks': 885,
 'total_tasksets': 295,
 'url': 'https://demo.lighttag.io/api/v1/projects/default/task_definitions/my-suggestions-task/'}


In [14]:
#Find your task by it's name
my_task_name = "My Suggestions Task" 
my_task = next(filter(lambda task: task['name'] == my_task_name,allTasks))

In [15]:

model

{'schema': 'e7e7de79-1623-4803-9aac-7e664f12117a',
 'name': 'My Newest LightTag Suggestions Model',
 'metadata': {'anything': 'here'},
 'tags': [{'id': '43ce3951-7ea1-423a-869f-ab07848d7f55',
   'name': 'PERSON',
   'description': ''},
  {'id': 'e332528b-dcb6-4414-9d14-fa61b12ab127',
   'name': 'LOCATION',
   'description': ''},
  {'id': '6e323f1d-1aa0-4e09-b64f-1ec0cd6dbed8',
   'name': 'ORGANIZATION',
   'description': ''},
  {'id': '978dfe2d-fdc3-4ca7-80d9-2baa0472f818',
   'name': 'TITLE',
   'description': ''}],
 'url': 'https://demo.lighttag.io/api/unstable/models/40f990db-b375-4292-8415-5621613b088f/'}

In [20]:
my_task_models = my_task['suggestion_models']
my_task_models.append(model['id']) # Add the id of the model you created to this task

session.put(my_task['url'],json={'models':my_task_models})

<Response [200]>

#### 5.3 Attaching a model when creating a task in the UI
Sometimes, you'll have predefined the model before starting a task. In that case you can attach the model to your task at creation time. 

1. Go to the New Task dialog
   ![newTask](./img/newTask.png)
   
2. Select the Schema your model belongs to 
    ![taskWithSchema](./img/taskWithSchema.png)
3. In the advanced tab, select your model
    ![taskWithModel](./img/taskWithSugModel.png)
